# Car and Dog Image Identifier

In [ ]:
#importing required Libraries

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import  Flatten,Dense,Dropout,Input
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Image Preporcessing

In [2]:
#This re scale the image pixels from 0-255 to 0-1
train_sc = ImageDataGenerator(rescale=1./255,rotation_range=20,          # Random rotation ±20 degrees
    width_shift_range=0.2,      # Horizontal shift
    height_shift_range=0.2,     # Vertical shift
    shear_range=0.2,            # Shear transformation
    zoom_range=0.2,             # Random zoom
    horizontal_flip=True,       # Flip images horizontally
    fill_mode='nearest'         # Fill new pixels after transformation
    )
test_sc  = ImageDataGenerator(rescale=1./255,rotation_range=20,          # Random rotation ±20 degrees
    width_shift_range=0.2,      # Horizontal shift
    height_shift_range=0.2,     # Vertical shift
    shear_range=0.2,            # Shear transformation
    zoom_range=0.2,             # Random zoom
    horizontal_flip=True,       # Flip images horizontally
    fill_mode='nearest'         # Fill new pixels after transformation
    )

#Load Training Images
train_data = train_sc.flow_from_directory(
    r"D:\DATA SCIENCE\DL\PROJECTS\car_bike\Car-Bike-Dataset\Training_Set", #path of the training data Folder
    target_size = (128,128), #resize all images into 64X64
    batch_size= 42,
    class_mode= "binary" #its binary model because (car and bike)
)

#Load test Images
test_data = test_sc.flow_from_directory(r"D:\DATA SCIENCE\DL\PROJECTS\car_bike\Car-Bike-Dataset\Test_Set",
                                       target_size=(128,128),
                                        batch_size = 42,
                                        class_mode = "binary"
                                       )
# Print class indices so you know exactly which is 0 and 1
print("Class indices:", train_data.class_indices)  # Example: {'Car': 0, 'Bike': 1}


Found 4000 images belonging to 2 classes.
Found 798 images belonging to 2 classes.
Class indices: {'Bike': 0, 'Car': 1}


# Build CNN Model

In [3]:
model = Sequential()

model.add(Input(shape=(128, 128, 3)))

# ---- Convolution Layer 1 ----
# Learns edges and simple shapes
model.add(Conv2D(filters = 32,kernel_size=(3,3),activation="relu",input_shape = (64,64,3)))

# Conv Layer 2 (NEW)
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Conv Layer 3 (NEW)
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# ---- Pooling Layer ----
# Reduces image size, keeps important info

model.add(MaxPooling2D(pool_size=(2,2)))

# ---- Convolution Layer 2 ----
# Learns more complex features
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatten
# Converts 2D feature maps into 1D vector

model.add(Flatten())
# Dense Layers with Dropout to reduce overfitting
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Helps prevent overfitting


# ---- Fully Connected Layer ----

model.add(Dense(1, activation='sigmoid'))



C:\Users\HP ELITE BOOK\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Compile the Model

In [4]:
model.compile(optimizer="adam",loss = "binary_crossentropy", metrics=['accuracy'])

# Train the Model

In [ ]:
model.fit(train_data,epochs=30,validation_data=test_data)

# Save the Model

In [ ]:
model.save("car_bike_cnn_model.h5")  # ✅ save as HDF5 file
model = load_model("car_bike_cnn_model.h5")

# Identify the Image

In [7]:
img_path = r"D:\DATA SCIENCE\DL\PROJECTS\car_bike\Car-Bike-Dataset\Test_Set\bike\Bike (10).png"

# Load and preprocess image
img = image.load_img(img_path, target_size=(128, 128))
img = image.img_to_array(img)
img = img / 255.0
img = np.expand_dims(img, axis=0)

# Predict
prob = model.predict(img)[0][0]

# Get class names
class_indices = train_data.class_indices
class_0 = [k for k, v in class_indices.items() if v == 0][0]
class_1 = [k for k, v in class_indices.items() if v == 1][0]

# Final prediction
if prob > 0.5:
    print(f"Prediction: {class_1}")
else:
    print(f"Prediction: {class_0}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: Car
